
# POC 激活验证

In [1]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP
from utils_gnn import unpadding, padding




# x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_0.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)
del params

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
del model_detail

import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

# get a image
image = None
label = None
for i, (x_in, y_in) in enumerate(dataloader):
    image = x_in
    model(image)
    label = y_in
    break
del trainset, dataloader

None
{'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}
None
{'kernel_size': 2, 'stride': 2, 'padding': 0, 'dilation': 1, 'ceil_mode': False}
None
None
None
1 in 32 out: torch.Size([1, 1, 28, 28])
conv1 weight size: torch.Size([32, 1, 5, 5])
32 in 32 out torch.Size([1, 32, 28, 28])
32 in 16 out: torch.Size([1, 32, 28, 28])
16 in 8 out: torch.Size([1, 16, 14, 14])


In [40]:
def my_conv(data, data_size, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # print(weight.size())
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size[0], kernel_size[1])
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    print("kernel_size, stride, padding:")
    print(kernel_size, stride, padding)
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, 
                    stride=stride, padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    print("conv input size:", data.size())
    x = operator(data.to("cuda"))

    return x


def maxpool(kernel_size, stride, padding):
    pass  

def message_func(edges):
    # print("this is message function.")
    # print("message fucntion ends")
    # print("edges src ft size:", edges.src['ft'].size())
    return {'m': edges.src['ft']}
    # return {}


def reduce(nodes):
    # print("this is reduce function")

    pre_data = nodes.mailbox['m']
    # print("mailbox data size:", pre_data.size())
    ft = torch.zeros((1, 1, 28, 28)).to("cuda")
    # print("nodes data size in reduce:", ft.size())
    # print("nodes ft size:", nodes.data['ft'].size())
    if pre_data.all() == 0:
        return {'h': torch.sum(pre_data, dim=1)}
    for i in range(len(pre_data)):
        # else do conv operation
        kernel_size, stride, padding = nodes.data['params'][i]

        res_ft = my_conv(pre_data[i][0], None, nodes.data['x'][i], nodes.data['bias'][i], kernel_size, stride, padding)
        # print("res_ft in reduce func:", res_ft)
        ft += res_ft
    # print("size before and after:", nodes.mailbox['m'].size(), ft.size())
    # print("reduce function ends")
    return {'h': ft}


def initiate_node_feature(graph, image):
    ft = None 
    mask = graph.ndata['tag'] == 0
    out_channels = int(sum(mask))
    ft = torch.zeros((len(graph.nodes()), 1, 1, 28, 28))
    convd_size = torch.zeros((len(graph.nodes()), 2))
    for i in range(out_channels):
        kernel_size, stride, padding = graph.ndata['params'][i]
        # do conv
        res_ft = my_conv(image, None, graph.ndata['x'][i], graph.ndata['bias'][i],
                kernel_size, stride, padding)
        # do relu
        relu_opt = torch.nn.functional.relu
        res_ft = relu_opt(res_ft)

        # do max pooling
        pooling = graph.ndata['pooling'][i]
        if pooling.all() != 0:
            # do max_pooling
            kernel_size, stride, pad, dilation, ceil_mode = pooling
            max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                padding=pad, dilation=dilation, ceil_mode=ceil_mode)
            
            res_ft = max_pooling_operator(res_ft)
        _, _, r, c = res_ft.size()
        ft[i] = res_ft
        convd_size[i] = torch.tensor([int(r), int(c)])
    graph.ndata['ft'] = ft.to("cuda")
    graph.ndata['convd_size'] = convd_size.to("cuda")


def conv_message_passing(graph):
    # get node id first, then get subgraph
    def nodes_feature_smaller_than_one(nodes):
        return nodes.data['tag'] <= 1
    
    nodes_idx = graph.filter_nodes(nodes_feature_smaller_than_one)
    # print("in conv mp, nodes idx:", nodes_idx)
    subg = dgl.node_subgraph(graph, nodes_idx)
    def conv_reduce_func():
        pass
    subg.update_all(dgl.function.copy_src('ft', 'm'))
    ft = subg.ndata['ft']

def cnn_cal(graph, image):
    initiate_node_feature(graph, image)
    conv_message_passing(graph)
    # graph.update_all(message_func=message_func, reduce_func=reduce_func)
    #ft = graph.ndata['ft'][0]
    #return ft

import json
# get cnn results
# pred, params = model(x_in)
# print(params.keys())
with open("./intermediate_data/params.json", "w") as f:
    json.dump(params, f)
# get gnn transmission results
with torch.no_grad():
    res = cnn_cal(g, image)


kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.Size([1, 1, 28, 28])
kernel_size, stride, padding:
tensor([5, 5], device='cuda:0') tensor([1, 1], device='cuda:0') tensor([2, 2], device='cuda:0')
conv input size: torch.S

TypeError: update_all() missing 1 required positional argument: 'reduce_func'

In [41]:
# def conv_reduce_func(nodes):
#     pass
# def reduce_func(nodes):
#     print("this is reduce function")

#     # input_mask = nodes.data['tag'] == 0
#     # conv_mask = nodes.data['tag'] == 1
#     # concat_mask = nodes.data['tag'] == 2
#     # fc_mask = nodes.data['tag'] == 3

#     # input_feat = nodes.data['x'][input_mask]
#     # print(input_feat)
#     pre_data = nodes.mailbox['m']
#     print("mailbox data size:", pre_data.size())
#     print("nodes data size in reduce:", nodes.data['x'].size())
#     ft = torch.zeros((28, 28)).to("cuda")
#     if pre_data.all() == 0:
#         return {'h': ft}
#     print("nodes ft size:", nodes.data['ft'].size())
#     for i in range(len(pre_data)):
#         # else do conv operation
#         kernel_size, stride, padding = nodes.data['params'][i]
#         res_ft = conv(pre_data[i][0], nodes.data['x'][i], nodes.data['bias'][i],
#                 kernel_size, stride, padding)
#         print("res_ft in reduce func:", res_ft)
#         # check if max pooling
#         ft += res_ft
#     print("reduce function ends")
#     return {'h': ft}

In [125]:
print(g.ndata['tag'])
# get node id
def nodes_with_feature_smaller_two(nodes):
    return nodes.data['tag'] <= 1
nodes_idx = g.filter_nodes(nodes_with_feature_smaller_two)
# print(nodes_idx)
subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
print(subg.nodes())
ft32 = None

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3],
       device='cuda:0')
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
       device='cuda:0')


In [126]:
subg.ndata['ft']

tensor([[[[[0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           ...,
           [0.1070, 0.1070, 0.0843,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.0515,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070]]]],



        [[[[0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           ...,
           [0.0206, 0.0206, 0.0000,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0108,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206]]]],



        [[[[0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635

In [127]:
subg.ndata['convd_size']

tensor([[28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],


In [128]:
print(subg.ndata['ft'])
print(subg.ndata['convd_size'])

tensor([[[[[0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           ...,
           [0.1070, 0.1070, 0.0843,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.0515,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070]]]],



        [[[[0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           ...,
           [0.0206, 0.0206, 0.0000,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0108,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206]]]],



        [[[[0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635

In [129]:
def do_conv():
    pass

global_ft = []
global_size = []
def my_reduce(nodes):
    global global_ft, global_size
    print("my reduce function called!")
    m = nodes.mailbox['m']
    if torch.eq(m, torch.zeros(m.size()).to("cuda")).all() == True:
        print("received message all are zeros! qaq")
    actual_size = nodes.mailbox['ret_size']
    global_ft.append(m)
    global_size.append(actual_size)
    
    print("mailbox data size:", m.size(), actual_size.size())
    print(nodes)
    mshape = m.size()
    n, f, d = actual_size.size()
    ret_size = torch.zeros((n, d)).to("cuda")
    ret_ft = torch.zeros((n, 1, 1, 28, 28)).to("cuda")
    pre_ft = None
    pre_weight = None
    cnt = 0 
    print("node bacht N: ", mshape)
    # reduce nodes batch N
    for out_idx in range(mshape[0]):
        # begin the process for each node
        received_feats = m[out_idx]
        if torch.eq(received_feats, torch.zeros(received_feats.size()).to("cuda")).all() == True:
            print("skiped %d feats." % cnt)
            cnt += 1
            continue
        received_sizes = actual_size[out_idx]
        # print("received_size: ", received_sizes)
        # a_break = input()
        # channels = received_feats.size()[0]
        ft = torch.zeros((1, 1, 28, 28)).to("cuda")
        # print("channels: %d " % channels)

        # prepare conv operator
        kernel_size, stride, pad = nodes.data['params'][out_idx]
        # print("idx:", out_idx, nodes.data['params'][out_idx])
        conv_operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, stride=stride, padding=pad)
        # get and set my_conv weight & bias
        weight = nodes.data['x'][out_idx]
        if pre_weight is None:
            pre_weight = weight
        elif torch.eq(pre_weight, weight).all() == True:
            print("this weight is same as pre weight totally!")
        else:
            print("weight diffs!")
        w = unpadding(weight, kernel_size[0], kernel_size[1])
        w = w.unsqueeze(0).unsqueeze(0)
        bias = nodes.data['bias'][out_idx]
        b = unpadding(bias, 1, 1)[0]
        # print("weight size:", conv_operator.weight.size())
        conv_operator.weight.data = w
        # print("weight size:", conv_operator.weight.size())
        conv_operator.bias.data = b
        row, col = -1, -1
        # for in_channel, data_size, size_idx in zip(received_feats, actual_size[out_idx], ret_size[0]):
        for inner_idx in range(len(received_feats)):
            in_channel = received_feats[inner_idx]
            data_size = received_sizes[inner_idx]
            # get actual size data
            row, col = data_size
            row, col = int(row), int(col)
            # print("types: ", type(row))
            # print("row: ", row)
            if row == 0:
                continue
            # resize 
            data = in_channel.resize_((28, 28))
            data = unpadding(data, row, col)
            data = data.resize_((1, 1, row, col))
        
            # do conv
            # print("input size:", data.size())
            data = conv_operator(data.to("cuda"))
            # print("finished conv, res size:", data.size())

            # do relu
            relu_operator = torch.nn.functional.relu
            data = relu_operator(data)

            # do max_pool
            pooling = nodes.data['pooling'][out_idx]
            if pooling.all() != 0:
                # do max_pooling
                kernel_size, stride, pad, dilation, ceil_mode = pooling
                max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                    padding=pad, dilation=dilation, ceil_mode=ceil_mode)
                
                data = max_pooling_operator(data)
            _, _, row, col = data.size()
            data = data.resize_((row, col))
            # print("maxPooling data size: ", data.size(), type(data))
            data = padding(data, 28, 28)
            if torch.eq(data, torch.zeros(data.size()).to("cuda")).all() == True:
                print("data is all zeros!")
            else:
                print("data feat seems ok!")
            ft += data
        convd_size = torch.tensor([row, col])
        ret_size[out_idx] = convd_size.to("cuda")
        ret_ft[out_idx] = ft.to("cuda")
        if torch.eq(ft, torch.zeros(ft.size()).to("cuda")).all() == True:
            print("idx: ", out_idx)
            # a_break = input()
        # else:
            # print(ft)
            # a_break = input()
        if pre_ft is None:
            pre_ft = ft
        elif torch.eq(pre_ft, ft).all() == True:
            print("this ft is equals pre ft totally!")
        else:
            print("final feat diffs!")
        pre_ft = ft
        # a_break = input()

    # return size is [n, 1, 1, 28, 28], reduced from [n, m, 1, 1, 28, 28]
    return {'h': ret_ft, 'ret_size': ret_size}

def my_message(edges):
    return {'m': edges.src['ft'], 'ret_size': edges.src['convd_size']}

def print_zeros_in_feat(key):
    print(key + ": ")
    zeros = []
    nonzero = []
    for id in subg.nodes():
        id = int(id)
        feat = subg.nodes[id].data[key]
        if torch.eq(feat, torch.zeros(feat.size()).to("cuda")).all() == True:
            zeros.append(id)
        else:
            nonzero.append(id)
    print("zeros - ", zeros)
    print("nonzeros - ", nonzero)


with torch.no_grad():
    subg.update_all(my_message, my_reduce)
    subg.ndata['convd_size'] = subg.ndata['ret_size']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    subg.ndata['ft'] += subg.ndata['h']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    cur_ft32 = subg.ndata['ft'][32]
    if ft32 is not None and torch.eq(cur_ft32, ft32).all():
        ft32 = cur_ft32
        print("ft32 equals!")
    else:
        ft32 = cur_ft32
        print("ft32 not equeal!")



my reduce function called!
received message all are zeros! qaq
mailbox data size: torch.Size([8, 16, 1, 1, 28, 28]) torch.Size([8, 16, 2])
node bacht N:  torch.Size([8, 16, 1, 1, 28, 28])
skiped 0 feats.
skiped 1 feats.
skiped 2 feats.
skiped 3 feats.
skiped 4 feats.
skiped 5 feats.
skiped 6 feats.
skiped 7 feats.
my reduce function called!
mailbox data size: torch.Size([48, 32, 1, 1, 28, 28]) torch.Size([48, 32, 2])
node bacht N:  torch.Size([48, 32, 1, 1, 28, 28])
skiped 0 feats.
skiped 1 feats.
skiped 2 feats.
skiped 3 feats.
skiped 4 feats.
skiped 5 feats.
skiped 6 feats.
skiped 7 feats.
skiped 8 feats.
skiped 9 feats.
skiped 10 feats.
skiped 11 feats.
skiped 12 feats.
skiped 13 feats.
skiped 14 feats.
skiped 15 feats.
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data feat seems ok!
data fe

In [122]:
subg.ndata['ft']

tensor([[[[[0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           ...,
           [0.1070, 0.1070, 0.0843,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.0515,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070]]]],



        [[[[0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           ...,
           [0.0206, 0.0206, 0.0000,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0108,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206]]]],



        [[[[0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635

In [110]:
for ft, st in zip(global_ft[1], global_size[1]):
    # print(len(global_ft[1]), len(global_size[1]))
    # print(ft, st)
    # check zeros
    condition = torch.eq(ft, torch.zeros(ft.size()).to("cuda")).all() == True
    if not condition:
        print("not all zeros!")
        print(ft)
        print(st)

not all zeros!
tensor([[[[[0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           ...,
           [0.1070, 0.1070, 0.0843,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.0515,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070]]]],



        [[[[0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           ...,
           [0.0206, 0.0206, 0.0000,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0108,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206]]]],



        [[[[0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
   

In [111]:
feats = []
for feat in subg.ndata['h']:
    feat = feat.cpu()
    if torch.eq(feat, torch.zeros(feat.size())).all() == True:
        feats.append("Zeros")
    else:
        feats.append("Feats")
print(feats)

['Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Zeros', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Feats', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros', 'Zeros']


In [9]:
print(subg.ndata['ft'].size())

torch.Size([88, 1, 1, 28, 28])


In [10]:
print(image)

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [11]:
cnn_params = None 
with open("./intermediate_data/params.json", "r") as f:
    cnn_params = json.load(f)
x_in = cnn_params['conv1']['in']

In [12]:
# input is the same
torch.eq(torch.tensor(x_in), image).all() == True

tensor(True)

In [112]:
ft = g.ndata['ft']
mask = g.ndata['tag'] == 0
print(ft[mask].size())
model_out = ft[mask]
out = torch.tensor(cnn_params['conv1']['out'])
print(out.size())
model_out = model_out.resize_((32, 28, 28)).to("cuda")
out = out.resize_((32, 28, 28)).to("cuda")
print(model_out.size(), out.size())

torch.Size([32, 1, 1, 28, 28])
torch.Size([1, 32, 28, 28])
torch.Size([32, 28, 28]) torch.Size([32, 28, 28])


In [113]:
# conv1 out is the same 
from torch.nn.functional import relu
torch.eq(model_out, relu(out)).all() == True

tensor(True, device='cuda:0')

In [130]:
# checking output by gnn
from torch.nn.functional import relu
conv2_out = torch.tensor(cnn_params['conv2']['out'])
print(conv2_out.size())
gnn2_out = subg.ndata['ft'][32:64]
print(gnn2_out.size())
conv2_out = conv2_out.resize_((32, 28, 28)).to("cuda")
gnn2_out = gnn2_out.resize_((32, 28, 28)).to("cuda")
print(conv2_out.size(), gnn2_out.size())
import numpy as np
for id in range(len(conv2_out)):
    np.savetxt("./intermediate_data/cnn_out_%d.csv" % id, conv2_out[id].cpu().numpy(), delimiter=',')
    np.savetxt("./intermediate_data/gnn_out_%d.csv" % id, gnn2_out[id].cpu().numpy(), delimiter=',')


torch.Size([1, 32, 28, 28])
torch.Size([32, 1, 1, 28, 28])
torch.Size([32, 28, 28]) torch.Size([32, 28, 28])
tensor(False, device='cuda:0')


In [118]:
print_zeros_in_feat('ft')

ft: 
zeros -  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 39, 40, 41, 42, 44, 47, 49, 50, 53, 56, 57, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
nonzeros -  [32, 34, 35, 37, 38, 43, 45, 46, 48, 51, 52, 54, 55, 58, 59, 60]


In [115]:
equals = False
for gf in gnn2_out:
    for cf in conv2_out:
        if torch.eq(gf, relu(cf)).all() == True:
            equeals = True
            break
        
print("equals: ", equals)

equals:  False


In [104]:
for g1 in gnn2_out:
    for g2 in gnn2_out:
        if torch.eq(g1, g2).all() == True:
            print("equals: ", True)

equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
equals:  True
